# 競馬予測_スクレイピング

### 0. colabの環境を整える

#### 0-1. git clone

In [ ]:
!git clone https://github.com/yuugo0724/keiba_prediction.git

#### 0-2. 作業ブランチの作成

In [ ]:
# cloneしたディレクトリに移動
%cd ./keiba_prediction

# ブランチ名がmainであること
!git branch
# 作業ブランチの作成
!git branch [作業ブランチ名]
# 作業ブランチにチェックアウト
!git checkout [作業ブランチ名]
# 作業ブランチにチェックアウトできていることを確認
!git branch

#### 0-2. ソースコードのディレクトリに移動

In [ ]:
%cd src/

#### 0-3. pythonのライブラリをインストール

In [ ]:
pip install -r ../dockerfile/requirements.txt

### 1. importと変数の定義

#### 1-1. import

In [1]:
"""
ライブラリ
"""
import pandas as pd
"""
モジュール(定数)
"""
# ローカルパス
from modules.constants import LocalPaths
# 学習データのパス
from modules.constants import TrainDataPaths
# データフレームの列名
from modules.constants import DataFrameCols
# レース名のマスター
from modules.constants import RaceInfo
"""
モジュール(前処理)
"""
# データフレームの作成
from modules.preprocess import create_dataframe
# スクレイピング
from modules.preprocess import scrapy_proc
# データフレームの整形
from modules.preprocess import dataframe_grades
"""
モジュール(モデルの学習)
"""
from modules.training import data_training

/home/keiba/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1-2. インスタンスの初期化

In [2]:
lps = LocalPaths()
df_cols = DataFrameCols()
race_info = RaceInfo()
# スクレイピング用クラスのインスタンス化
scrapy_proc = scrapy_proc(lps)
# マスタ作成用クラスのインスタンス化
create_dataframe = create_dataframe(lps, df_cols)
# 学習データのパスクラスをインスタンス化
#   モデルのタイプ
#     multiclass_3
#     multiclass_5
#     binaryclass ※上手く動かせない・・・
#   カテゴリ変数の処理方法を決める
#     ラベルエンコーディング　：label
#     ダミー変数化　　　　　　：dummies ※列が多くなりすぎるため廃止
#     カウントエンコーディング：count  ※競馬予測に向いていない
tdp = TrainDataPaths('binaryclass','label')

### 2. スクレイピング
1. スクレイピング対象のurlを取得  
2. レース結果データを取得  
3. 馬ごとの成績データを取得  
  ※学習データとしては、まだ利用していない  
4. 血統データを取得  
  ※学習データとしては、まだ利用していない  

#### 2-1. スクレイピング対象のurl取得

In [ ]:
# urlを取得する日付のレンジを指定
df_gen_date = pd.date_range(start="20221201",end="20221231", freq="MS")
# フォーマットを変えてリストに格納
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
race_date_list = df_date.values

# スクレイピング対象のurlをスクレイピング
scrapy_proc.coll_urls(race_date_list)

#### 2-2. レース結果のパスリスト作成

In [ ]:
# urlファイルのパスリストを定義
scrapy_proc.create_path_list()

#### 2-3. レース結果の取得

In [ ]:
# レース結果を取得する日付のレンジを指定
df_gen_date = pd.date_range(start="20221201",end="20221231", freq="MS")
# フォーマットを変えてリストに格納
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
race_date_list = df_date.values

# レース結果対象のurlパスリスト作成
scrapy_proc.create_race_path_list(race_date_list)
# レース結果のスクレイピング
scrapy_proc.coll_grades()

#### 2-4. レース結果マスターの作成

In [ ]:
# レース結果を取得する日付のレンジを指定
df_gen_date = pd.date_range(start="20140101",end="20221231", freq="MS")
# フォーマットを変えてリストに格納
df_date = df_gen_date.to_series().dt.strftime("%Y%m")
race_date_list = df_date.values

create_dataframe.create_master_grades(race_date_list)

#### 2-6. レース結果マスターの読み込み

In [ ]:
grades_master = pd.read_pickle(lps.DATA_GRADES_MASTER, compression='zip')

#### 2-7. 馬IDマスターの作成
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
create_dataframe.create_master_horseid(grades_master)

#### 2-8. 騎手IDマスターの作成

In [ ]:
create_dataframe.create_master_jockeyid(grades_master)

#### 2-9. 調教師IDマスターの作成

In [ ]:
create_dataframe.create_master_trainerid(grades_master)

#### 【未実装】2-10. 馬ごとのレース結果を取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]
#horse_id_list = df_horse_id_master.values

scrapy_proc.coll_horse_grades(horse_id_list)

#### 【未実装】2-11. 血統データを取得
※現状学習データに含めるつもりはないので実施不要  
　今後、学習データに含める場合にコードを修正

In [ ]:
horse_id_list = ["2018104963","2018105074"]
#horse_id_list = df_horse_id_master.values

scrapy_proc.coll_predigree(horse_id_list)

#### 重賞一覧取得

In [ ]:
# レース結果を取得する日付のレンジを指定
df_gen_date = pd.date_range(start="2012",end="2022", freq="MS")
# フォーマットを変えてリストに格納
df_date = df_gen_date.to_series().dt.strftime("%Y")
df_date = df_date.drop_duplicates()
juushou_date_list = df_date.values

# レース結果のスクレイピング
scrapy_proc.coll_juushou(juushou_date_list,lps.DATA_JUUSHOU_LIST)

### 3. 前処理

#### 3-1. 前処理に必要なクラスのインスタンス化

In [ ]:
# レース結果マスタを読み込み
grades_master = pd.read_pickle(lps.DATA_GRADES_MASTER, compression='zip')
# データフレームの処理クラスのインスタンス化
grades = dataframe_grades(grades_master, lps, tdp, df_cols, race_info)

#### 3-2. レース情報を分割し、下記列を追加
- 距離
- 回り
- タイプ
- 馬場状態

In [ ]:
grades.train_split_raceinfo()

#### G1・G2・G3レース列を追加

In [ ]:
grades.add_G_race()

#### 3-3. レース結果マスタのデータ整形

In [ ]:
# 競馬場名を指定
# 例：中山
place_name = None
#place_name = '中山'

# 欠損値の削除
grades.remove_missing_values()
# 体重増減の記号(+)を削除
grades.conversion_values()
# 性齢を性と齢で分割
grades.split_sexual_age()
# 競馬場指定(デフォルト指定なし)
grades.select_place_name(place_name)
# レースタイプを指定(デフォルト指定なし)
# 芝・ダート・障
# 引数はリスト
race_type_list = ['芝','ダート']
grades.select_race_type(race_type_list)
# スクレイピングの不具合で周りに「障」が含まれているため、その削除
#grades.tmp_del_shougai()
# 整数型に型変換
grades.conversion_int()

#### 3-4. レース結果マスタから目的変数用に着順を変更
モデルのタイプ  
- multiclass_3  
  １着　：0  
  ２着　：1  
  ３着　：2  
  その他：3  
- multiclass_5  
  １着　：0  
  ２着　：1  
  ３着　：2  
  ４着　：3  
  ５着　：4  
  その他：5  
- binaryclass ※上手く動かせない・・・  
  １着　：0  
  その他：1  

In [ ]:
grades.train_data_model_type()

#### 3-5. カテゴリ変数(馬名・騎手・調教師)の処理
共通で処理するカテゴリ変数  
- 回り
- タイプ
- 性

In [ ]:
grades.category_process()

#### 3-6. 学習データの前処理・保存
【前処理】  
1. アンダーサンプリング  
2. ~~標準化~~  
  ※決定木では不要なため実施しない  

【保存】  
- 説明変数  
- 目的変数  
- 学習データ  
- 検証データ  
- LightGBMのデータセット  

In [ ]:
grades.data_preprocess()

In [ ]:
grades.output_grades_master()

### 4. モデルの学習

#### 4-1. クラスのインスタンス化

In [3]:
data_train = data_training(lps, tdp, df_cols)

#### 4-2. モデルの学習

In [4]:
data_train.train_lgb()

[I 2023-01-04 07:05:54,277] A new study created in memory with name: no-name-c7a861b1-fcf2-45fe-b044-58acd69de6e5
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/keiba/.local/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/keiba/.local/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/home/keiba/.local/lib/python3.9/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_resu

[LightGBM] [Info] Number of positive: 122013, number of negative: 61035
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015907 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1003
[LightGBM] [Info] Number of data points in the train set: 183048, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666563 -> initscore=0.692680
[LightGBM] [Info] Start training from score 0.692680
Training until validation scores don't improve for 10 rounds
[10]	train's binary_logloss: 0.607498	valid's binary_logloss: 0.608296
[20]	train's binary_logloss: 0.595448	valid's binary_logloss: 0.597113
[30]	train's binary_logloss: 0.589746	valid's binary_logloss: 0.592176
[40]	train's binary_logloss: 0.584948	valid's binary_logloss: 0.588108
[50]	train's binary_logloss: 0.582079	valid's binary_logloss: 0.585961
[60]	train's binary_logloss: 0.580055	valid's binary_logloss: 0.584633
[70]	train's bin

feature_fraction, val_score: 0.576729:  14%|#4        | 1/7 [00:05<00:30,  5.05s/it]

[LightGBM] [Info] Number of positive: 122013, number of negative: 61035
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002029 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1003
[LightGBM] [Info] Number of data points in the train set: 183048, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666563 -> initscore=0.692680
[LightGBM] [Info] Start training from score 0.692680
Training until validation scores don't improve for 10 rounds
[10]	train's binary_logloss: 0.605541	valid's binary_logloss: 0.606485
[20]	train's binary_logloss: 0.594601	valid's binary_logloss: 0.596468
[30]	train's binary_logloss: 0.588851	valid's binary_logloss: 0.5915
[40]	train's binary_logloss: 0.584487	valid's binary_logloss: 0.587812
[50]	train's binary_logloss: 0.581783	valid's binary_logloss: 0.58585
[60]	train's binary_loglos

feature_fraction, val_score: 0.575815:  29%|##8       | 2/7 [00:13<00:35,  7.18s/it][I 2023-01-04 07:06:08,010] Trial 1 finished with value: 0.5758150967970606 and parameters: {'feature_fraction': 0.6}. Best is trial 1 with value: 0.5758150967970606.


...

[240]	train's binary_logloss: 0.498916	valid's binary_logloss: 0.57179
[250]	train's binary_logloss: 0.496765	valid's binary_logloss: 0.571814
Early stopping, best iteration is:
[241]	train's binary_logloss: 0.498753	valid's binary_logloss: 0.571784
[LightGBM] [Info] Number of positive: 122013, number of negative: 61035
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1003
[LightGBM] [Info] Number of data points in the train set: 183048, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666563 -> initscore=0.692680
[LightGBM] [Info] Start training from score 0.692680
Training until validation scores don't improve for 10 rounds
[10]	train's binary_logloss: 0.598193	valid's binary_logloss: 0.602422
[20]	train's binary_logloss: 0.5825	valid's binary_loglos

min_data_in_leaf, val_score: 0.571385:  80%|########  | 4/5 [00:40<00:10, 10.52s/it]

[280]	train's binary_logloss: 0.490307	valid's binary_logloss: 0.571488
Early stopping, best iteration is:
[278]	train's binary_logloss: 0.490826	valid's binary_logloss: 0.571451
[LightGBM] [Info] Number of positive: 122013, number of negative: 61035
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1003
[LightGBM] [Info] Number of data points in the train set: 183048, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666563 -> initscore=0.692680
[LightGBM] [Info] Start training from score 0.692680
Training until validation scores don't improve for 10 rounds
[10]	train's binary_logloss: 0.598227	valid's binary_logloss: 0.602458
[20]	train's binary_logloss: 0.582529	valid's binary_logloss: 0.59084
[30]	train's binary_logloss: 0.572842	valid's binary_loglo

min_data_in_leaf, val_score: 0.571385: 100%|##########| 5/5 [00:49<00:00,  9.85s/it]

[210]	train's binary_logloss: 0.505774	valid's binary_logloss: 0.571534
[220]	train's binary_logloss: 0.50364	valid's binary_logloss: 0.571549
Early stopping, best iteration is:
[212]	train's binary_logloss: 0.505287	valid's binary_logloss: 0.571505


#### 4-3. モデルのパラメータを確認

In [ ]:
# パラメータをロード
data_train.load_parameter()

In [ ]:
# パラメータを確認
data_train.show_parameter()

In [ ]:
# 混合配列を確認
data_train.show_mixed_array()

In [ ]:
# 分類レポートを確認
data_train.show_report()

#### 4-4. 学習推移を確認

In [ ]:
data_train.show_graph()

In [ ]:
# 頻度とゲインを表示
# 引数
#  頻度：'split'
#  ゲイン：'gain'
data_train.show_feature_importance('split')

In [ ]:
# 頻度とゲインを表示
# 引数
#  頻度：'split'
#  ゲイン：'gain'
data_train.show_feature_importance('gain')

### 5. 予測

#### 5-1. 予測クラスのインスタンス化

In [ ]:
# 学習データのパスクラスをインスタンス化
#   モデルのタイプ
#     multiclass_3
#     multiclass_5
#     binaryclass
#   カテゴリ変数の処理方法を決める
#     ラベルエンコーディング　：pred_label
#     ダミー変数化　　　　　　：pred_dummies
#     カウントエンコーディング：pred_count
pred_tdp = TrainDataPaths('multiclass_5','pred_label')

#### 5-2. 予測するレースurlをセット
既に着順が出ているレースはエラーになる

In [ ]:
#pred_url = 'https://race.netkeiba.com/race/shutuba.html?race_id=202206050811&rf=race_list'

pred_url = 'https://race.netkeiba.com/race/result.html?race_id=202209060711&rf=race_list'

#### 5-3. 予測レースのスクレイピング

In [ ]:
scrapy_proc.coll_pred_grades(pred_url)

In [ ]:
scrapy_proc.coll_eval_grades(pred_url)

#### 5-4. 前処理に必要なクラスのインスタンス化

In [ ]:
# レース結果マスタを読み込み
pred_keiba = pd.read_pickle(lps.DATA_TMP_PRED)
# データフレームの処理クラスのインスタンス化
predict = dataframe_grades(pred_keiba, lps, pred_tdp, df_cols, race_info)

#### 5-5. レース情報を分割し、下記列を追加
- 距離
- 回り
- タイプ
- 馬場状態

In [ ]:
predict.pred_split_raceinfo()

#### G1・G2・G3レース列を追加

In [ ]:
predict.add_G_race()

#### 5-6. 予測レース結果を整形

In [ ]:
# 競馬場名を指定
# 例：中山
place_name = None

# 体重増減の記号(+)を削除
predict.pred_conversion_values()
# 性齢を性と齢で分割
predict.split_sexual_age()
# 競馬場指定(デフォルト指定なし)
predict.select_place_name(place_name)
# レースタイプを指定(デフォルト指定なし)
# 芝・ダート・障
# 引数はリスト
race_type_list = ['芝','ダート']
predict.select_race_type(race_type_list)
# 空白列を0で埋める
predict.blank_conversion()
#grades.tmp_del_shougai()
# 整数型に型変換
predict.pred_conversion_int()

#### 5-7. カテゴリ変数(馬名・騎手・調教師)の処理
共通で処理するカテゴリ変数  
- 回り
- タイプ
- 性

In [ ]:
predict.category_process()

#### 5-8. 学習データのダミー変数化した列をマージ

In [ ]:
predict.pred_merge_dummies()

#### 5-9. 予測データに空白列が含まれることがあるので削除

In [ ]:
predict.blank_conversion()

#### 5-10. 予測データを読み込み

In [ ]:
pred_data = predict.grades_master

### 【確認用】予測データのデータ型を確認

In [ ]:
predict.show_grades_master()

#### 【確認用】予測データをcsv出力

In [ ]:
predict.grades_master.to_csv('/home/keiba/src/data/tmp/pred_race_grades.csv')

#### 5-11. モデル用クラスをインスタンス化

In [ ]:
data_pred = data_training(lps, pred_tdp, df_cols)

#### 5-12. 予測結果を表示・json出力

In [ ]:
data_pred.model_predict_detail(pred_data)